## Canonical Skeleton Matching

This notebook is organized into three stages:

1. Stage 1: Canonical skeleton extraction
2. Stage 2: Canonical skeleton visualization (for raw and averaged skeletons, optional)
3. Stage 3: Live matching, rep tracking, and voice feedback

Files Required to Get Started: exercises_data/bicep_curl/up, exercises_data/bicep_curl/down and main.ipynb

Quick start:
- Run Stage 1 to generate `up.json` and `down.json` from your dataset.
- Use Stage 2 to preview canonical skeletons.
- Run the averaging step to produce `up_averaged.json` and `down_averaged.json`.
- Run Stage 3 for the live, voice-guided bicep curl tracker.

### Stage 1: Extraction of canonical skeletons  
* It automatically filters out non-visible keypoints based on MediaPipe’s landmark visibility score.  
* It stores only the coordinates of visible joints in each image.  
* Each exercise can have a different set of landmarks based on what is visible in its sample images.  
  
How this helps:  
* For half-body exercises, like bicep curls, it might only store shoulders, elbows, wrists, and hips.  
* For full-body exercises, like Russian twists, it will include all visible points from head to ankles.  
* The system will still use hip midpoint normalization to keep consistency across exercises.  

In [1]:
!pip install opencv-python mediapipe numpy

In [10]:
import os
import json
import cv2
import mediapipe as mp
import numpy as np

# Setting up MediaPipe Pose for static image processing
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)

# Config for filtering and normalization
# I skip face/head landmarks from Pose
FACE_KEYPOINTS = set(range(0, 11))
VISIBILITY_THRESHOLD = 0.8  # same threshold

# Helper functions I reuse below

def get_reference_normalized_landmarks(landmarks):
    """
    Normalise landmarks based on hip midpoint as origin and
    shoulder–hip distance as scale.
    Excludes face/head keypoints and low-visibility landmarks.
    """
    left_hip = np.array([landmarks[23].x, landmarks[23].y, landmarks[23].z])
    right_hip = np.array([landmarks[24].x, landmarks[24].y, landmarks[24].z])
    origin = (left_hip + right_hip) / 2.0

    left_shoulder = np.array([landmarks[11].x, landmarks[11].y, landmarks[11].z])
    right_shoulder = np.array([landmarks[12].x, landmarks[12].y, landmarks[12].z])
    shoulder_center = (left_shoulder + right_shoulder) / 2.0

    scale = np.linalg.norm(shoulder_center - origin)
    if scale < 1e-6:
        scale = 1.0

    normalized = {}
    for idx, lm in enumerate(landmarks):
        if idx in FACE_KEYPOINTS:
            continue
        if lm.visibility < VISIBILITY_THRESHOLD:
            continue

        x, y, z = np.array([lm.x, lm.y, lm.z]) - origin
        normalized[str(idx)] = {
            'x': float(x / scale),
            'y': float(y / scale),
            'z': float(z / scale),
            'visibility': float(lm.visibility)
        }
    return normalized


def extract_landmarks_from_image(image_path):
    """Extract normalised landmarks from a single image."""
    image = cv2.imread(image_path)
    if image is None:
        return None

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)
    if not results.pose_landmarks:
        return None

    return get_reference_normalized_landmarks(results.pose_landmarks.landmark)


def save_to_json(file_path, exercise_name, data):
    """Append new exercise data to the corresponding stage JSON file."""
    if os.path.exists(file_path):
        with open(file_path, 'r') as f:
            all_data = json.load(f)
    else:
        all_data = {}

    if exercise_name not in all_data:
        all_data[exercise_name] = []

    all_data[exercise_name].extend(data)

    with open(file_path, 'w') as f:
        json.dump(all_data, f, indent=4)


# Main extraction pipeline

def process_exercises(base_dir='exercises_data'):
    """
    Traverse exercise folders and extract pose landmarks.
    Each exercise should contain subfolders: up, down.
    """
    stage_files = {
        'up': 'up.json',
        'down': 'down.json'
    }

    for exercise in os.listdir(base_dir):
        exercise_path = os.path.join(base_dir, exercise)
        if not os.path.isdir(exercise_path):
            continue

        print(f"\n🏋️ Processing exercise: {exercise}")

        for stage, json_file in stage_files.items():
            stage_path = os.path.join(exercise_path, stage)
            if not os.path.exists(stage_path):
                continue

            print(f"  📸 Stage: {stage}")
            stage_data = []

            for img_name in os.listdir(stage_path):
                if not img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                    continue

                img_path = os.path.join(stage_path, img_name)
                landmarks = extract_landmarks_from_image(img_path)

                if landmarks and len(landmarks) > 0:
                    stage_data.append(landmarks)

            if stage_data:
                save_to_json(json_file, exercise, stage_data)
                print(f"    ✅ Saved {len(stage_data)} frames to {json_file}")
            else:
                print(f"    ⚠️ No valid frames found for {stage}")

    print("\n✅ Canonical skeleton extraction complete!")


if __name__ == "__main__":
    process_exercises()



🏋️ Processing exercise: bicep_curl
  📸 Stage: up
    ✅ Saved 4 frames to up.json
  📸 Stage: down
    ✅ Saved 5 frames to down.json

✅ Canonical skeleton extraction complete!


### Stage 2: Visualizing canonical skeletons  
* Code 1: for up.json and down.json visualization  
* Code 2: for up_averaged.json and down_averaged.json visualization  
  
This script:  
* Reads stored JSON files (up.json, down.json)  
* Extracts canonical skeletons for each exercise  
* Displays them on a black background  
* Saves one .png per sample in a folder (canonical_previews_raw)  

#### Visualizing canonical skeletons (Code 1)

Render raw canonical samples from `up.json` and `down.json`. Save previews to `canonical_previews_raw/`.

In [11]:
# Code 1: visualize raw canonical stages

import os
import json
import cv2
import numpy as np
import mediapipe as mp

# Setting up MediaPipe Pose drawing
mp_pose = mp.solutions.pose
POSE_CONNECTIONS = mp_pose.POSE_CONNECTIONS

# Inputs and preview output
JSON_FILES = ["up.json", "down.json"]
OUTPUT_FOLDER = "canonical_previews_raw"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)


def normalize_and_center(points, image_size=720, padding=0.1):
    """Scale and center points so the entire skeleton fits nicely in the frame."""
    points = np.array(points)
    min_xy = points.min(axis=0)
    max_xy = points.max(axis=0)
    center = (max_xy + min_xy) / 2
    scale = (1 - 2 * padding) * image_size / max(max_xy - min_xy)

    normalized = (points - center) * scale + image_size / 2
    return normalized.astype(int)


def draw_skeleton(image, landmarks_dict, color_joint=(0, 255, 0), color_line=(255, 255, 128)):
    """Draw canonical skeleton clearly within image bounds."""
    coords = []
    for idx, lm in landmarks_dict.items():
        coords.append([lm["x"], lm["y"]])
    coords = np.array(coords)

    # Normalize and center points in-frame
    coords = normalize_and_center(coords)
    idx_list = list(landmarks_dict.keys())
    points = {int(idx_list[i]): tuple(coords[i]) for i in range(len(coords))}

    # Draw limb connections
    for start_idx, end_idx in POSE_CONNECTIONS:
        if start_idx in points and end_idx in points:
            cv2.line(image, points[start_idx], points[end_idx], color_line, 2)

    # Draw joint circles
    for pt in points.values():
        cv2.circle(image, pt, 5, color_joint, -1)

    return image


def visualize_json(json_file):
    """Display all skeletons from one JSON file neatly within the frame."""
    if not os.path.exists(json_file):
        print(f"⚠️ {json_file} not found, skipping...")
        return

    with open(json_file, "r") as f:
        data = json.load(f)

    stage = os.path.splitext(os.path.basename(json_file))[0]
    print(f"\n🎯 Visualizing stage: {stage}")

    for exercise_name, skeletons in data.items():
        print(f"  ➜ Exercise: {exercise_name}")
        for idx, skeleton in enumerate(skeletons):
            img = np.zeros((720, 720, 3), dtype=np.uint8)
            img = draw_skeleton(img, skeleton)
            cv2.putText(img, f"{exercise_name.upper()} - {stage.upper()}",
                        (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.9,
                        (255, 255, 255), 2, cv2.LINE_AA)

            filename = f"{exercise_name}_{stage}_{idx + 1}.png"
            filepath = os.path.join(OUTPUT_FOLDER, filename)
            cv2.imwrite(filepath, img)

    print(f"✅ Saved all visualizations for {stage} to '{OUTPUT_FOLDER}'")


def main():
    for json_file in JSON_FILES:
        visualize_json(json_file)


if __name__ == "__main__":
    main()



🎯 Visualizing stage: up
  ➜ Exercise: bicep_curl
✅ Saved all visualizations for up to 'canonical_previews_raw'

🎯 Visualizing stage: down
  ➜ Exercise: bicep_curl
✅ Saved all visualizations for down to 'canonical_previews_raw'


#### Additional stage: Averaging canonical skeletons  
Store the average keypoints from all collected data in up.json and down.json for each exercise. This helps to have just one image per stage for each exercise.  

Script:  
* Reads the existing up.json and down.json, which contain multiple samples per exercise, and calculates the average position for each landmark. It then writes up_averaged.json and down_averaged.json.  

It computes the mean for each joint across all collected samples and writes:  
- up_averaged.json  
- down_averaged.json  

In [12]:
import json, os, numpy as np

# Stages I average
stages = ["up", "down"]

def average_skeletons(stage_file):
    """Average landmark coordinates across multiple samples per exercise."""
    with open(stage_file, "r") as f:
        data = json.load(f)

    averaged = {}
    for exercise, samples in data.items():
        # Grab all landmark keys seen across samples
        all_keys = set().union(*[set(s.keys()) for s in samples])
        exercise_avg = {}

        for key in all_keys:
            coords = np.array([[s[key]["x"], s[key]["y"], s[key]["z"]] 
                               for s in samples if key in s])
            if len(coords) > 0:
                mean_vals = np.mean(coords, axis=0)
                exercise_avg[key] = {"x": float(mean_vals[0]),
                                     "y": float(mean_vals[1]),
                                     "z": float(mean_vals[2])}
        averaged[exercise] = exercise_avg
    return averaged


for stage in stages:
    input_file = f"{stage}.json"
    output_file = f"{stage}_averaged.json"
    if os.path.exists(input_file):
        avg_data = average_skeletons(input_file)
        with open(output_file, "w") as f:
            json.dump(avg_data, f, indent=4)
        print(f"✅ Averaged {stage} → {output_file}")
    else:
        print(f"⚠️ Missing {input_file}, skipping...")


✅ Averaged up → up_averaged.json
✅ Averaged down → down_averaged.json


#### Visualizing averaged skeletons (Code 2)

Render averaged canonical skeletons from `up_averaged.json` and `down_averaged.json`. Save previews to `canonical_previews_averaged/`.

In [13]:
# Code 2: visualize averaged canonical stages

import os
import json
import cv2
import numpy as np
import mediapipe as mp

# Setting up MediaPipe Pose drawing
mp_pose = mp.solutions.pose
POSE_CONNECTIONS = mp_pose.POSE_CONNECTIONS

# Inputs and preview output
JSON_FILES = ["up_averaged.json", "down_averaged.json"]
OUTPUT_FOLDER = "canonical_previews_averaged"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)


def normalize_and_center(points, image_size=720, padding=0.1):
    """Scale and center points so the entire skeleton fits nicely in the frame."""
    points = np.array(points)
    min_xy = points.min(axis=0)
    max_xy = points.max(axis=0)
    center = (max_xy + min_xy) / 2
    scale = (1 - 2 * padding) * image_size / max(max_xy - min_xy)

    normalized = (points - center) * scale + image_size / 2
    return normalized.astype(int)


def draw_skeleton(image, landmarks_dict, color_joint=(0, 255, 0), color_line=(255, 255, 128)):
    """Draw canonical skeleton clearly within image bounds."""
    coords = []
    for idx, lm in landmarks_dict.items():
        coords.append([lm["x"], lm["y"]])
    coords = np.array(coords)

    # Normalize and center points in-frame
    coords = normalize_and_center(coords)
    idx_list = list(landmarks_dict.keys())
    points = {int(idx_list[i]): tuple(coords[i]) for i in range(len(coords))}

    # Draw limb connections
    for start_idx, end_idx in POSE_CONNECTIONS:
        if start_idx in points and end_idx in points:
            cv2.line(image, points[start_idx], points[end_idx], color_line, 2)

    # Draw joint circles
    for pt in points.values():
        cv2.circle(image, pt, 5, color_joint, -1)

    return image


def visualize_json(json_file):
    """Display all skeletons from one JSON file neatly within the frame."""
    if not os.path.exists(json_file):
        print(f"⚠️ {json_file} not found, skipping...")
        return

    with open(json_file, "r") as f:
        data = json.load(f)

    stage = os.path.splitext(os.path.basename(json_file))[0]
    print(f"\n🎯 Visualizing stage: {stage}")

    for exercise_name, skeletons in data.items():
        print(f"  ➜ Exercise: {exercise_name}")
        if isinstance(skeletons, dict):
            skeleton_list = [skeletons]  # single averaged skeleton
        else:
            skeleton_list = skeletons

        for idx, skeleton in enumerate(skeleton_list):
            img = np.zeros((720, 720, 3), dtype=np.uint8)
            img = draw_skeleton(img, skeleton)
            cv2.putText(img, f"{exercise_name.upper()} - {stage.upper()}",
                        (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.9,
                        (255, 255, 255), 2, cv2.LINE_AA)

            filename = f"{exercise_name}_{stage}_{idx + 1}.png"
            filepath = os.path.join(OUTPUT_FOLDER, filename)
            cv2.imwrite(filepath, img)

    print(f"✅ Saved all visualizations for {stage} to '{OUTPUT_FOLDER}'")


def main():
    for json_file in JSON_FILES:
        visualize_json(json_file)


if __name__ == "__main__":
    main()



🎯 Visualizing stage: up_averaged
  ➜ Exercise: bicep_curl
✅ Saved all visualizations for up_averaged to 'canonical_previews_averaged'

🎯 Visualizing stage: down_averaged
  ➜ Exercise: bicep_curl
✅ Saved all visualizations for down_averaged to 'canonical_previews_averaged'


### Stage 3: Main logic (Live Matching & Scaling Script):

* Opens the webcam and detects pose in real time using MediaPipe.
* Loads the averaged skeleton for the selected exercise or stage.
* Matches poses based on angles and adjusts scaling for each user. It scales each limb segment—such as shoulder to elbow, elbow to wrist, hip to knee—so the distances fit the user's proportions.
* Automatically moves up, down, and then up again when the user closely matches each stage.
* Tracks repetitions and provides voice feedback.
* Does not move to the next stage while giving feedback for adjustments, but continues during motivational messages.

In [16]:
# angle_based_bicep_feedback_with_voice_fixed_modified_V2.py
import cv2
import json
import numpy as np
import mediapipe as mp
import time
import threading
from math import acos, degrees
from gtts import gTTS
import playsound
import tempfile
import os
import random

# ======================
# Voice helpers
# ======================
voice_lock = threading.Lock()
is_speaking = False
is_feedback_speaking = False # Track when I'm speaking adjustment vs motivation

def speak(text, is_adjustment=False):
    """Speak text asynchronously using gTTS. Sets is_feedback_speaking for adjustments."""
    global is_speaking, is_feedback_speaking

    def _play():
        global is_speaking, is_feedback_speaking
        with voice_lock:
            if is_speaking:
                return
            is_speaking = True
            is_feedback_speaking = is_adjustment # Set flag only for adjustments
            
        try:
            tts = gTTS(text=text, lang="en", tld="co.uk")
            temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".mp3")
            filename = temp_file.name
            temp_file.close()
            tts.save(filename)
            playsound.playsound(filename)
            os.remove(filename)
        except Exception as e:
            print("Speech error:", e)
        finally:
            with voice_lock:
                is_speaking = False
                is_feedback_speaking = False # Clear flag when finished

    threading.Thread(target=_play, daemon=True).start()

def trainer_feedback(counter):
    motivational = [
        "Great work, keep going!",
        "Nice form, stay strong!",
        "Push harder, you got this!",
        "Excellent! Keep it up!",
        "Stay focused, don't quit!"
    ]
    if counter % 5 == 0:
        return f"{counter}! {random.choice(motivational)}"
    else:
        return str(counter)

# ===========================
# Pose setup and core config
# ===========================
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
POSE_CONNECTIONS = mp_pose.POSE_CONNECTIONS
pose_detector = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

ANGLE_THRESH_DEG = {"elbow": 35.0, "shoulder": 35.0, "wrist": 35.0, "torso": 35.0}
REP_MIN_DURATION = 0.5
MATCH_HOLD_FRAMES = 6

L = {
    "L_SHOULDER": 11, "R_SHOULDER": 12,
    "L_ELBOW": 13, "R_ELBOW": 14,
    "L_WRIST": 15, "R_WRIST": 16,
    "L_HIP": 23, "R_HIP": 24
}

STAGES = ["up", "down"]
EXERCISE = "bicep_curl"

# Map joint indices to angle keys (used for green highlights)
JOINT_TO_ANGLE_MAP = {
    L["L_ELBOW"]: "L_elbow", L["R_ELBOW"]: "R_elbow",
    L["L_SHOULDER"]: "L_shoulder", L["R_SHOULDER"]: "R_shoulder",
    L["L_WRIST"]: "L_wrist", L["R_WRIST"]: "R_wrist"
}

# ======================
# Geometry helpers
# ======================
def calculate_angle(a, b, c):
    a, b, c = np.array(a[:2]), np.array(b[:2]), np.array(c[:2])
    ba, bc = a - b, c - b
    denom = np.linalg.norm(ba) * np.linalg.norm(bc)
    if denom < 1e-8:
        return None
    cosang = np.clip(np.dot(ba, bc) / denom, -1.0, 1.0)
    return degrees(acos(cosang))

def torso_angle_deg(user_mid_sh, user_mid_hip):
    v = user_mid_sh - user_mid_hip
    if np.linalg.norm(v) < 1e-6:
        return 0.0
    vert = np.array([0.0, -1.0])
    cosang = np.clip(np.dot(v, vert) / (np.linalg.norm(v)*np.linalg.norm(vert)), -1.0, 1.0)
    return degrees(acos(cosang))

def load_averaged(stage_name):
    fname = f"{stage_name}_averaged.json"
    try:
        with open(fname, "r") as f:
            data = json.load(f)
        return data.get(EXERCISE, {})
    except Exception as e:
        print("Error loading", fname, e)
        return {}

def canonical_angles_from_points(canon_points):
    angles = {}
    def get(i): return np.array([canon_points[str(i)]["x"], canon_points[str(i)]["y"]]) if str(i) in canon_points else None
    L_sh, R_sh = get(L["L_SHOULDER"]), get(L["R_SHOULDER"])
    L_el, R_el = get(L["L_ELBOW"]), get(L["R_ELBOW"])
    L_wr, R_wr = get(L["L_WRIST"]), get(L["R_WRIST"])
    L_hip, R_hip = get(L["L_HIP"]), get(L["R_HIP"])
    if L_sh is not None and L_el is not None and L_wr is not None:
        angles["L_elbow"] = calculate_angle(L_sh, L_el, L_wr)
    if R_sh is not None and R_el is not None and R_wr is not None:
        angles["R_elbow"] = calculate_angle(R_sh, R_el, R_wr)
    if L_el is not None and L_sh is not None and L_hip is not None:
        angles["L_shoulder"] = calculate_angle(L_el, L_sh, L_hip)
    if R_el is not None and R_sh is not None and R_hip is not None:
        angles["R_shoulder"] = calculate_angle(R_el, R_sh, R_hip)
    if L_el is not None and L_wr is not None and L_sh is not None:
        angles["L_wrist"] = calculate_angle(L_el, L_wr, L_sh)
    if R_el is not None and R_wr is not None and R_sh is not None:
        angles["R_wrist"] = calculate_angle(R_el, R_wr, R_sh)
    if L_sh is not None and R_sh is not None and L_hip is not None and R_hip is not None:
        canon_mid_sh = (L_sh + R_sh)/2
        canon_mid_hip = (L_hip + R_hip)/2
        v = canon_mid_sh - canon_mid_hip
        vert = np.array([0.0, -1.0])
        if np.linalg.norm(v) > 1e-8:
            cosang = np.clip(np.dot(v, vert)/(np.linalg.norm(v)*np.linalg.norm(vert)),-1.0,1.0)
            angles["torso"] = degrees(acos(cosang))
    return angles

def compute_user_angles(lm):
    angles = {}
    def Lm(i): return np.array([lm[i].x, lm[i].y]) if i < len(lm) else None
    L_sh, R_sh = Lm(L["L_SHOULDER"]), Lm(L["R_SHOULDER"])
    L_el, R_el = Lm(L["L_ELBOW"]), Lm(L["R_ELBOW"])
    L_wr, R_wr = Lm(L["L_WRIST"]), Lm(L["R_WRIST"])
    L_hip, R_hip = Lm(L["L_HIP"]), Lm(L["R_HIP"])
    if L_sh is not None and L_el is not None and L_wr is not None:
        angles["L_elbow"] = calculate_angle(L_sh, L_el, L_wr)
    if R_sh is not None and R_el is not None and R_wr is not None:
        angles["R_elbow"] = calculate_angle(R_sh, R_el, R_wr)
    if L_el is not None and L_sh is not None and L_hip is not None:
        angles["L_shoulder"] = calculate_angle(L_el, L_sh, L_hip)
    if R_el is not None and R_sh is not None and R_hip is not None:
        angles["R_shoulder"] = calculate_angle(R_el, R_sh, R_hip)
    if L_el is not None and L_wr is not None and L_sh is not None:
        angles["L_wrist"] = calculate_angle(L_el, L_wr, L_sh)
    if R_el is not None and R_wr is not None and R_sh is not None:
        angles["R_wrist"] = calculate_angle(R_el, R_wr, R_sh)
    if L_sh is not None and R_sh is not None and L_hip is not None and R_hip is not None:
        user_mid_sh = (L_sh + R_sh)/2
        user_mid_hip = (L_hip + R_hip)/2
        angles["torso"] = torso_angle_deg(user_mid_sh, user_mid_hip)
    return angles

def angle_match_mask(canon_angles_stage, user_angles):
    mask, diffs = {}, []
    for k, can_ang in canon_angles_stage.items():
        ua = user_angles.get(k)
        if ua is None: 
            mask[k] = False
            continue
        diff = abs(can_ang - ua)
        diffs.append(diff)
        thr = ANGLE_THRESH_DEG.get(k.split("_")[-1], 25.0)
        mask[k] = diff <= thr
    return mask, np.mean(diffs) if diffs else 999.0

def get_directional_feedback(user_pt, canon_pt, joint_name):
    dx, dy = canon_pt[0] - user_pt[0], canon_pt[1] - user_pt[1]
    if abs(dx) < 0.01 and abs(dy) < 0.01:
        return ""
    if abs(dy) > abs(dx):
        dir_msg = "lift up" if dy < 0 else "lower down"
    else:
        dir_msg = "move inward" if dx < 0 else "move outward"
    
    # Swap L_/R_ for mirrored camera display
    if joint_name.startswith("L_"):
        swapped_name = "R_" + joint_name[2:] # L_ becomes R_
    elif joint_name.startswith("R_"):
        swapped_name = "L_" + joint_name[2:] # R_ becomes L_
    else:
        swapped_name = joint_name
    # end swap note

    joint_name_display = swapped_name.replace("L_", "Left ").replace("R_", "Right ")
    return f"Adjust your {joint_name_display.lower().replace('_',' ')}: {dir_msg}"

# =====================
# Main loop
# =====================
canonical_data = {s: load_averaged(s) for s in STAGES}
canonical_angles = {s: canonical_angles_from_points(canonical_data[s]) for s in STAGES}

cap = cv2.VideoCapture(0)
print("📷 Starting camera... Press 'q' to quit")

rep_count = 0
current_stage_idx = 0
direction = 1
stable_match_frames = 0
last_rep_time = None
dir_feedback = ""
spoken_feedback = ""
display_feedback = ""
spoken_rep = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose_detector.process(img_rgb)
    display = cv2.addWeighted(frame, 0.55, np.zeros_like(frame), 0.45, 0)

    stage_name = STAGES[current_stage_idx]
    canon_pts = canonical_data.get(stage_name, {})
    canon_angles_stage = canonical_angles.get(stage_name, {})

    if results.pose_landmarks:
        lm = results.pose_landmarks.landmark
        user_angles = compute_user_angles(lm)
        match_mask, mean_diff = angle_match_mask(canon_angles_stage, user_angles)

        total_joints = len(match_mask)
        matched_joints = sum(1 for m in match_mask.values() if m)
        match_ratio = matched_joints / total_joints if total_joints else 0
        allow_feedback = match_ratio >= 0.8 # only speak adjustments when most joints are aligned

        # Align canonical to the user (scale + rotate)
        def align_canonical(canon_pts_dict):
            cpts = {int(k): np.array([v["x"], v["y"]]) for k,v in canon_pts_dict.items()}
            try:
                cLsh, cRsh = cpts[L["L_SHOULDER"]], cpts[L["R_SHOULDER"]]
                cLhip, cRhip = cpts[L["L_HIP"]], cpts[L["R_HIP"]]
            except: return cpts
            canon_mid_sh, canon_mid_hip = (cLsh+cRsh)/2, (cLhip+cRhip)/2
            user_L_sh = np.array([lm[L["L_SHOULDER"]].x, lm[L["L_SHOULDER"]].y])
            user_R_sh = np.array([lm[L["R_SHOULDER"]].x, lm[L["R_SHOULDER"]].y])
            user_L_hip = np.array([lm[L["L_HIP"]].x, lm[L["L_HIP"]].y])
            user_R_hip = np.array([lm[L["R_HIP"]].x, lm[L["R_HIP"]].y])
            user_mid_sh = (user_L_sh + user_R_sh)/2
            user_mid_hip = (user_L_hip + user_R_hip)/2
            c_vec, u_vec = canon_mid_sh - canon_mid_hip, user_mid_sh - user_mid_hip
            scale = np.linalg.norm(u_vec)/(np.linalg.norm(c_vec)+1e-8)
            ang = np.arctan2(u_vec[1],u_vec[0]) - np.arctan2(c_vec[1],c_vec[0])
            R = np.array([[np.cos(ang), -np.sin(ang)],[np.sin(ang), np.cos(ang)]])
            aligned = {}
            for i,p in cpts.items():
                shifted=(p-canon_mid_hip)*scale
                rotated=R.dot(shifted)
                aligned[i]=rotated+user_mid_hip
            return aligned

        aligned_canon = align_canonical(canon_pts)

        # Feedback logic
        new_feedback = ""
        is_adjustment_feedback = False # Flag to mark if this is an adjustment feedback
        
        if allow_feedback:
            for key, idx in {
                "L_elbow": L["L_ELBOW"], "R_elbow": L["R_ELBOW"],
                "L_shoulder": L["L_SHOULDER"], "R_shoulder": L["R_SHOULDER"],
                "L_wrist": L["L_WRIST"], "R_wrist": L["R_WRIST"]
            }.items():
                # If this angle misses, nudge the user
                if key in match_mask and not match_mask[key]:
                    if idx in aligned_canon:
                        canon_p = aligned_canon[idx]
                        user_p = np.array([lm[idx].x, lm[idx].y])
                        new_feedback = get_directional_feedback(user_p, canon_p, key)
                        is_adjustment_feedback = True
                        break

        # Only speak the latest feedback for this frame
        if allow_feedback and new_feedback and new_feedback != spoken_feedback:
            if not is_speaking:
                # Pass the is_adjustment flag to the speak function
                speak(new_feedback, is_adjustment=is_adjustment_feedback)
                spoken_feedback = new_feedback
                display_feedback = new_feedback
        elif not new_feedback and not is_speaking:
            spoken_feedback = ""
            display_feedback = ""

        # === Draw skeletons with green matched joints ===
        # 1) Draw limb connections
        for (s, e) in POSE_CONNECTIONS:
            if s in aligned_canon and e in aligned_canon:
                p1 = (int(aligned_canon[s][0]*w), int(aligned_canon[s][1]*h))
                p2 = (int(aligned_canon[e][0]*w), int(aligned_canon[e][1]*h))
                
                # light blue/white looks clean over the dark background
                color = (200, 200, 255) 

                cv2.line(display, p1, p2, color, 2)

        # 2) Draw key joints; green when the corresponding angle matches
        for idx, p in aligned_canon.items():
            angle_key = JOINT_TO_ANGLE_MAP.get(idx)
            
            # Check if the angle associated with this joint is matched
            joint_matched = match_mask.get(angle_key, False) if angle_key else False

            # green when matched, otherwise red/blue
            color = (0, 255, 0) if joint_matched else (0, 0, 255) # Green if matched, Red/Blue otherwise
            
            cv2.circle(display, (int(p[0]*w), int(p[1]*h)), 6, color, -1)


        # Stage switching: only advance when I’m not speaking an adjustment
        all_matched = all(match_mask.values()) if match_mask else False
        
        if not is_feedback_speaking: # Only proceed if not saying specific adjustment feedback
            if all_matched:
                stable_match_frames += 1
            else:
                stable_match_frames = 0

            if stable_match_frames >= MATCH_HOLD_FRAMES:
                prev_stage = current_stage_idx
                current_stage_idx += direction
                current_stage_idx = np.clip(current_stage_idx, 0, len(STAGES)-1)
                
                if current_stage_idx in [0, len(STAGES)-1]:
                    direction *= -1
                
                if STAGES[current_stage_idx] == "up":
                    now = time.time()
                    last_rep_duration = (now - last_rep_time) if last_rep_time else None
                    last_rep_time = now
                    rep_count += 1
                    
                    # Give motivational feedback. This is separate and does NOT block stage progression.
                    if not new_feedback and not is_speaking:
                        speak(trainer_feedback(rep_count))
                    spoken_rep = rep_count
                
                stable_match_frames = 0

        # UI overlay
        cv2.putText(display, f"BICEP CURL - {STAGES[current_stage_idx].upper()}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(display, f"REPS: {rep_count}", (10, 70),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)
        # cv2.putText(display, f"ANG.DIFF: {mean_diff:.1f}°", (10, 105),
        #             cv2.FONT_HERSHEY_SIMPLEX, 0.7, (200, 200, 200), 2)
        if display_feedback:
            cv2.putText(display, display_feedback, (10, h - 40),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

    cv2.imshow("Bicep Curl - Directional Feedback", display)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

📷 Starting camera... Press 'q' to quit


C:\Users\chara\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\chara\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\chara\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\google\protobuf\symbol_database.py:

## Capstone: Gesture-driven workout flow (optional)

- Opens the camera and shows a simple menu. 
- The user selects options by holding a finger pose for about 1.5 seconds. 
- Uses start and stop hand gestures to control rep tracking during the exercise and to provide a rest or warm-up period. 
- Returns to the menu after completing the exercise and displays remaining options.

In [26]:
import cv2
import json
import numpy as np
import mediapipe as mp
import time
import threading
from math import acos, degrees
from gtts import gTTS
import playsound
import tempfile
import os
import random

# ------------------------------
# App config
# ------------------------------
FPS = 30
HOLD_FRAMES = int(1 * FPS)   # ~1 second hold
MAX_HANDS = 2
WINDOW_W, WINDOW_H = 800, 620   # Medium size

# Single-exercise flow for now (Bicep Curl)
exercises = ["Bicep Curl"]

# ======================
# Voice helpers
# ======================
voice_lock = threading.Lock()
is_speaking = False
is_feedback_speaking = False
last_adjustment_ts = 0.0
ADJUST_COOLDOWN_SEC = 1.2

def speak(text, is_adjustment=False):
    """Speak text asynchronously using gTTS."""
    global is_speaking, is_feedback_speaking, last_adjustment_ts

    def _play():
        global is_speaking, is_feedback_speaking
        with voice_lock:
            if is_speaking:
                return
            is_speaking = True
            is_feedback_speaking = is_adjustment
            if is_adjustment:
                last_adjustment_ts = time.time()
        try:
            tts = gTTS(text=text, lang="en", tld="co.uk")
            temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".mp3")
            filename = temp_file.name
            temp_file.close()
            tts.save(filename)
            playsound.playsound(filename)
            os.remove(filename)
        except Exception as e:
            print("Speech error:", e)
        finally:
            with voice_lock:
                is_speaking = False
                is_feedback_speaking = False

    threading.Thread(target=_play, daemon=True).start()

def trainer_feedback(counter):
    motivational = [
        "Great work, keep going!",
        "Nice form, stay strong!",
        "Push harder, you got this!",
        "Excellent! Keep it up!",
        "Stay focused, don't quit!"
    ]
    if counter % 5 == 0:
        return f"{counter}! {random.choice(motivational)}"
    else:
        return str(counter)

# ===========================
# Pose setup and core config
# ===========================
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
POSE_CONNECTIONS = mp_pose.POSE_CONNECTIONS
hands_detector = mp_hands.Hands(max_num_hands=MAX_HANDS,
                                min_detection_confidence=0.6,
                                min_tracking_confidence=0.6)
pose_detector = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

ANGLE_THRESH_DEG = {"elbow": 35.0, "shoulder": 35.0, "wrist": 35.0, "torso": 35.0}
MATCH_HOLD_FRAMES = 6

L = {
    "L_SHOULDER": 11, "R_SHOULDER": 12,
    "L_ELBOW": 13, "R_ELBOW": 14,
    "L_WRIST": 15, "R_WRIST": 16,
    "L_HIP": 23, "R_HIP": 24
}

STAGES = ["up", "down"]
EXERCISE = "bicep_curl"

JOINT_TO_ANGLE_MAP = {
    L["L_ELBOW"]: "L_elbow", L["R_ELBOW"]: "R_elbow",
    L["L_SHOULDER"]: "L_shoulder", L["R_SHOULDER"]: "R_shoulder",
    L["L_WRIST"]: "L_wrist", L["R_WRIST"]: "R_wrist"
}

# ======================
# Geometry + UI helpers
# ======================
def calculate_angle(a, b, c):
    a, b, c = np.array(a[:2]), np.array(b[:2]), np.array(c[:2])
    ba, bc = a - b, c - b
    denom = np.linalg.norm(ba) * np.linalg.norm(bc)
    if denom < 1e-8:
        return None
    cosang = np.clip(np.dot(ba, bc) / denom, -1.0, 1.0)
    return degrees(acos(cosang))

def torso_angle_deg(user_mid_sh, user_mid_hip):
    v = user_mid_sh - user_mid_hip
    if np.linalg.norm(v) < 1e-6:
        return 0.0
    vert = np.array([0.0, -1.0])
    cosang = np.clip(np.dot(v, vert) / (np.linalg.norm(v)*np.linalg.norm(vert)), -1.0, 1.0)
    return degrees(acos(cosang))

def load_averaged(stage_name):
    fname = f"{stage_name}_averaged.json"
    try:
        with open(fname, "r") as f:
            data = json.load(f)
        return data.get(EXERCISE, {})
    except Exception as e:
        print("Error loading", fname, e)
        return {}

def canonical_angles_from_points(canon_points):
    angles = {}
    def get(i): return np.array([canon_points[str(i)]["x"], canon_points[str(i)]["y"]]) if str(i) in canon_points else None
    L_sh, R_sh = get(L["L_SHOULDER"]), get(L["R_SHOULDER"])
    L_el, R_el = get(L["L_ELBOW"]), get(L["R_ELBOW"])
    L_wr, R_wr = get(L["L_WRIST"]), get(L["R_WRIST"])
    L_hip, R_hip = get(L["L_HIP"]), get(L["R_HIP"])
    if L_sh is not None and L_el is not None and L_wr is not None:
        angles["L_elbow"] = calculate_angle(L_sh, L_el, L_wr)
    if R_sh is not None and R_el is not None and R_wr is not None:
        angles["R_elbow"] = calculate_angle(R_sh, R_el, R_wr)
    if L_el is not None and L_sh is not None and L_hip is not None:
        angles["L_shoulder"] = calculate_angle(L_el, L_sh, L_hip)
    if R_el is not None and R_sh is not None and R_hip is not None:
        angles["R_shoulder"] = calculate_angle(R_el, R_sh, R_hip)
    if L_el is not None and L_wr is not None and L_sh is not None:
        angles["L_wrist"] = calculate_angle(L_el, L_wr, L_sh)
    if R_el is not None and R_wr is not None and R_sh is not None:
        angles["R_wrist"] = calculate_angle(R_el, R_wr, R_sh)
    if L_sh is not None and R_sh is not None and L_hip is not None and R_hip is not None:
        canon_mid_sh = (L_sh + R_sh)/2
        canon_mid_hip = (L_hip + R_hip)/2
        v = canon_mid_sh - canon_mid_hip
        vert = np.array([0.0, -1.0])
        if np.linalg.norm(v) > 1e-8:
            cosang = np.clip(np.dot(v, vert)/(np.linalg.norm(v)*np.linalg.norm(vert)),-1.0,1.0)
            angles["torso"] = degrees(acos(cosang))
    return angles

def compute_user_angles(lm):
    angles = {}
    def Lm(i): return np.array([lm[i].x, lm[i].y]) if i < len(lm) else None
    L_sh, R_sh = Lm(L["L_SHOULDER"]), Lm(L["R_SHOULDER"])
    L_el, R_el = Lm(L["L_ELBOW"]), Lm(L["R_ELBOW"])
    L_wr, R_wr = Lm(L["L_WRIST"]), Lm(L["R_WRIST"])
    L_hip, R_hip = Lm(L["L_HIP"]), Lm(L["R_HIP"])
    if L_sh is not None and L_el is not None and L_wr is not None:
        angles["L_elbow"] = calculate_angle(L_sh, L_el, L_wr)
    if R_sh is not None and R_el is not None and R_wr is not None:
        angles["R_elbow"] = calculate_angle(R_sh, R_el, R_wr)
    if L_el is not None and L_sh is not None and L_hip is not None:
        angles["L_shoulder"] = calculate_angle(L_el, L_sh, L_hip)
    if R_el is not None and R_sh is not None and R_hip is not None:
        angles["R_shoulder"] = calculate_angle(R_el, R_sh, R_hip)
    if L_el is not None and L_wr is not None and L_sh is not None:
        angles["L_wrist"] = calculate_angle(L_el, L_wr, L_sh)
    if R_el is not None and R_wr is not None and R_sh is not None:
        angles["R_wrist"] = calculate_angle(R_el, R_wr, R_sh)
    if L_sh is not None and R_sh is not None and L_hip is not None and R_hip is not None:
        user_mid_sh = (L_sh + R_sh)/2
        user_mid_hip = (L_hip + R_hip)/2
        angles["torso"] = torso_angle_deg(user_mid_sh, user_mid_hip)
    return angles

def angle_match_mask(canon_angles_stage, user_angles):
    mask, diffs = {}, []
    for k, can_ang in canon_angles_stage.items():
        ua = user_angles.get(k)
        if ua is None: 
            mask[k] = False
            continue
        diff = abs(can_ang - ua)
        diffs.append(diff)
        thr = ANGLE_THRESH_DEG.get(k.split("_")[-1], 25.0)
        mask[k] = diff <= thr
    return mask, np.mean(diffs) if diffs else 999.0

def get_directional_feedback(user_pt, canon_pt, joint_name):
    dx, dy = canon_pt[0] - user_pt[0], canon_pt[1] - user_pt[1]
    if abs(dx) < 0.01 and abs(dy) < 0.01:
        return ""
    if abs(dy) > abs(dx):
        dir_msg = "lift up" if dy < 0 else "lower down"
    else:
        dir_msg = "move inward" if dx < 0 else "move outward"
    
    if joint_name.startswith("L_"):
        swapped_name = "R_" + joint_name[2:]
    elif joint_name.startswith("R_"):
        swapped_name = "L_" + joint_name[2:]
    else:
        swapped_name = joint_name

    joint_name_display = swapped_name.replace("L_", "Left ").replace("R_", "Right ")
    return f"Adjust your {joint_name_display.lower().replace('_',' ')}: {dir_msg}"



# ------------------------------
# Text rendering helper
# ------------------------------
def draw_centered_text(frame, text, y, max_width_frac=0.92, base_scale=1.2, thickness=3, color=(255,255,255)):
    h, w = frame.shape[:2]
    max_w = int(w * max_width_frac)
    scale = base_scale
    while scale > 0.3:
        (tw, th), _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, scale, thickness)
        if tw <= max_w:
            x = (w - tw) // 2
            cv2.putText(frame, text, (x, y), cv2.FONT_HERSHEY_SIMPLEX, scale, color, thickness)
            return
        scale -= 0.05
    (tw, th), _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1)
    cv2.putText(frame, text, ((w - tw) // 2, y), cv2.FONT_HERSHEY_SIMPLEX, 0.35, color, 1)

# ------------------------------
# Simple finger counting using relative landmark positions
# ------------------------------
def count_fingers_single(landmarks, hand_label="Right"):
    fingers_open = 0
    if hand_label == "Right":
        if landmarks[4].x < landmarks[3].x:
            fingers_open += 1
    else:
        if landmarks[4].x > landmarks[3].x:
            fingers_open += 1

    finger_tips = [8, 12, 16, 20]
    finger_pips = [6, 10, 14, 18]

    for tip, pip in zip(finger_tips, finger_pips):
        if landmarks[tip].y < landmarks[pip].y:
            fingers_open += 1

    return fingers_open

def total_finger_count(results):
    total = 0
    if not results.multi_hand_landmarks or not results.multi_handedness:
        return 0
    for hand_landmarks, hand_handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
        hand_label = hand_handedness.classification[0].label
        total += count_fingers_single(hand_landmarks.landmark, hand_label)
    return total

def detect_start_single(landmarks, hand_label="Right"):
    return count_fingers_single(landmarks, hand_label) >= 4

def detect_stop_single(landmarks, hand_label="Right"):
    return count_fingers_single(landmarks, hand_label) == 0

# ------------------------------
# Load averaged canonical data
# ------------------------------
canonical_data = {s: load_averaged(s) for s in STAGES}
canonical_angles = {s: canonical_angles_from_points(canonical_data[s]) for s in STAGES}

# ------------------------------
# App state
# ------------------------------
phase = "menu"
pending_index = None
selected_exercise = None
exercise_running = False
selection_hold = 0
start_hold = 0
stop_hold = 0

# Bicep curl exercise state
rep_count = 0
current_stage_idx = 0
direction = 1
stable_match_frames = 0
last_rep_time = None
spoken_feedback = ""
display_feedback = ""
spoken_rep = 0

# ------------------------------
# Video loop
# ------------------------------
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("Cannot open camera")

cv2.namedWindow("Workout Tracker", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Workout Tracker", WINDOW_W, WINDOW_H)

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.flip(frame, 1)
        h, w, _ = frame.shape
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        hands_results = hands_detector.process(rgb)

        # -------- menu --------
        if phase == "menu":
            draw_centered_text(frame,
                               "Hold 1 finger for 1 second to select Bicep Curl",
                               40, base_scale=0.9, thickness=2, color=(0, 255, 0))

            x_base, y_base, line_h = 40, 100, 46
            for i, ex in enumerate(exercises, start=1):
                y = y_base + (i-1) * line_h
                color = (0, 255, 0)
                if pending_index == i-1:
                    cv2.rectangle(frame, (x_base-12, y-30), (x_base+500, y+10), (255, 0, 0), -1)
                    color = (255, 255, 255)
                cv2.putText(frame, f"{i}. {ex}", (x_base, y),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

            if hands_results.multi_hand_landmarks:
                for hand_landmarks in hands_results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                effective_count = total_finger_count(hands_results)
                if effective_count == 1:  # single exercise → look for one raised finger
                    if pending_index != 0:
                        pending_index = 0
                        selection_hold = 1
                    else:
                        selection_hold += 1
                else:
                    pending_index = None
                    selection_hold = 0

                if pending_index is not None:
                    bar_x1, bar_y1 = 40, h-50
                    bar_x2, bar_y2 = w-40, h-20
                    progress = min(1.0, selection_hold / HOLD_FRAMES)
                    cv2.rectangle(frame, (bar_x1, bar_y1), (bar_x2, bar_y2), (255, 255, 255), 2)
                    cv2.rectangle(frame, (bar_x1, bar_y1),
                                  (int(bar_x1 + progress*(bar_x2-bar_x1)), bar_y2),
                                  (0, 255, 0), -1)

                if pending_index is not None and selection_hold >= HOLD_FRAMES:
                    selected_exercise = exercises[pending_index]
                    phase = "exercise"
                    exercise_running = False
                    pending_index = None
                    selection_hold = 0
                    # Reset bicep curl state
                    rep_count = 0
                    current_stage_idx = 0
                    direction = 1
                    stable_match_frames = 0
                    last_rep_time = None
                    spoken_feedback = ""
                    display_feedback = ""
                    spoken_rep = 0
            else:
                pending_index = None
                selection_hold = 0

        # -------- EXERCISE PHASE --------
        elif phase == "exercise":
            # Handle start/stop gestures
            if hands_results.multi_hand_landmarks:
                first_landmarks = hands_results.multi_hand_landmarks[0].landmark
                first_hand_label = hands_results.multi_handedness[0].classification[0].label

                if not exercise_running:
                    if detect_start_single(first_landmarks, first_hand_label):
                        start_hold += 1
                    else:
                        start_hold = 0
                    if start_hold >= HOLD_FRAMES:
                        exercise_running = True
                        start_hold = 0
            else:
                start_hold = 0
                stop_hold = 0

            # If exercise is running, process bicep curl
            if exercise_running:
                # Check stop gesture without interrupting pose tracking
                if hands_results.multi_hand_landmarks:
                    first_landmarks = hands_results.multi_hand_landmarks[0].landmark
                    first_hand_label = hands_results.multi_handedness[0].classification[0].label
                    if detect_stop_single(first_landmarks, first_hand_label):
                        stop_hold += 1
                    else:
                        stop_hold = 0
                    if stop_hold >= HOLD_FRAMES:
                        exercise_running = False
                        stop_hold = 0
                        selected_exercise = None
                        phase = "menu"
                        # Reset bicep curl state
                        rep_count = 0
                        current_stage_idx = 0
                        direction = 1
                        stable_match_frames = 0
                        last_rep_time = None
                        spoken_feedback = ""
                        display_feedback = ""
                        spoken_rep = 0
                        time.sleep(0.2)
                        continue

                
                pose_results = pose_detector.process(rgb)
                # display = cv2.addWeighted(frame, 0.55, np.zeros_like(frame), 0.45, 0)
                display = cv2.addWeighted(frame, 0.7, np.zeros_like(frame), 0.2, 0)
                

                stage_name = STAGES[current_stage_idx]
                canon_pts = canonical_data.get(stage_name, {})
                canon_angles_stage = canonical_angles.get(stage_name, {})

                if pose_results.pose_landmarks:
                    lm = pose_results.pose_landmarks.landmark
                    user_angles = compute_user_angles(lm)
                    match_mask, mean_diff = angle_match_mask(canon_angles_stage, user_angles)

                    total_joints = len(match_mask)
                    matched_joints = sum(1 for m in match_mask.values() if m)
                    match_ratio = matched_joints / total_joints if total_joints else 0
                    allow_feedback = match_ratio >= 0.8

                    # Align canonical skeleton - nested function like in Cell 12
                    def align_canonical(canon_pts_dict):
                        cpts = {int(k): np.array([v["x"], v["y"]]) for k,v in canon_pts_dict.items()}
                        try:
                            cLsh, cRsh = cpts[L["L_SHOULDER"]], cpts[L["R_SHOULDER"]]
                            cLhip, cRhip = cpts[L["L_HIP"]], cpts[L["R_HIP"]]
                        except: return cpts
                        canon_mid_sh, canon_mid_hip = (cLsh+cRsh)/2, (cLhip+cRhip)/2
                        user_L_sh = np.array([lm[L["L_SHOULDER"]].x, lm[L["L_SHOULDER"]].y])
                        user_R_sh = np.array([lm[L["R_SHOULDER"]].x, lm[L["R_SHOULDER"]].y])
                        user_L_hip = np.array([lm[L["L_HIP"]].x, lm[L["L_HIP"]].y])
                        user_R_hip = np.array([lm[L["R_HIP"]].x, lm[L["R_HIP"]].y])
                        user_mid_sh = (user_L_sh + user_R_sh)/2
                        user_mid_hip = (user_L_hip + user_R_hip)/2
                        c_vec, u_vec = canon_mid_sh - canon_mid_hip, user_mid_sh - user_mid_hip
                        scale = np.linalg.norm(u_vec)/(np.linalg.norm(c_vec)+1e-8)
                        ang = np.arctan2(u_vec[1],u_vec[0]) - np.arctan2(c_vec[1],c_vec[0])
                        R = np.array([[np.cos(ang), -np.sin(ang)],[np.sin(ang), np.cos(ang)]])
                        aligned = {}
                        for i,p in cpts.items():
                            shifted=(p-canon_mid_hip)*scale
                            rotated=R.dot(shifted)
                            aligned[i]=rotated+user_mid_hip
                        return aligned

                    aligned_canon = align_canonical(canon_pts)

                    # Feedback logic with cooldown to reduce delay and spam
                    new_feedback = ""
                    is_adjustment_feedback = False

                    now_ts = time.time()
                    can_prompt_adjustment = (now_ts - last_adjustment_ts) >= ADJUST_COOLDOWN_SEC

                    if allow_feedback and can_prompt_adjustment:
                        for key, idx in {
                            "L_elbow": L["L_ELBOW"], "R_elbow": L["R_ELBOW"],
                            "L_shoulder": L["L_SHOULDER"], "R_shoulder": L["R_SHOULDER"],
                            "L_wrist": L["L_WRIST"], "R_wrist": L["R_WRIST"]
                        }.items():
                            if key in match_mask and not match_mask[key]:
                                if idx in aligned_canon:
                                    canon_p = aligned_canon[idx]
                                    user_p = np.array([lm[idx].x, lm[idx].y])
                                    new_feedback = get_directional_feedback(user_p, canon_p, key)
                                    is_adjustment_feedback = True
                                    break

                    # Speak only current frame's feedback
                    if allow_feedback and new_feedback and new_feedback != spoken_feedback:
                        if not is_speaking:
                            speak(new_feedback, is_adjustment=is_adjustment_feedback)
                            spoken_feedback = new_feedback
                            display_feedback = new_feedback
                    elif not new_feedback and not is_speaking:
                        spoken_feedback = ""
                        display_feedback = ""

                    # === Draw skeletons with green matched joints ===
                    # 1. Draw connections
                    for (s, e) in POSE_CONNECTIONS:
                        if s in aligned_canon and e in aligned_canon:
                            p1 = (int(aligned_canon[s][0]*w), int(aligned_canon[s][1]*h))
                            p2 = (int(aligned_canon[e][0]*w), int(aligned_canon[e][1]*h))
                            
                            # Connection color is light blue/white
                            color = (200, 200, 255)

                            cv2.line(display, p1, p2, color, 2)

                    # 2. Draw key joints (circles) and check for angle match for the green color
                    for idx, p in aligned_canon.items():
                        angle_key = JOINT_TO_ANGLE_MAP.get(idx)
                        
                        # Check if the angle associated with this joint is matched
                        joint_matched = match_mask.get(angle_key, False) if angle_key else False

                        # Set color based on match status
                        color = (0, 255, 0) if joint_matched else (0, 0, 255)  # Green if matched, Red/Blue otherwise
                        
                        cv2.circle(display, (int(p[0]*w), int(p[1]*h)), 6, color, -1)

                    # Stage switching logic — allow progression when fully matched (even if adjustment speech is ongoing)
                    all_matched = all(match_mask.values()) if match_mask else False

                    if all_matched:
                        stable_match_frames += 1
                    else:
                        stable_match_frames = 0

                    if stable_match_frames >= MATCH_HOLD_FRAMES:
                        prev_stage = current_stage_idx
                        current_stage_idx += direction
                        current_stage_idx = np.clip(current_stage_idx, 0, len(STAGES)-1)
                        
                        if current_stage_idx in [0, len(STAGES)-1]:
                            direction *= -1
                        
                        if STAGES[current_stage_idx] == "up":
                            now = time.time()
                            last_rep_duration = (now - last_rep_time) if last_rep_time else None
                            last_rep_time = now
                            rep_count += 1
                            
                            # Motivational callouts don’t block stage changes
                            if not new_feedback and not is_speaking:
                                speak(trainer_feedback(rep_count))
                            spoken_rep = rep_count
                        
                        stable_match_frames = 0

                    # UI Info
                    cv2.putText(display, f"BICEP CURL - {STAGES[current_stage_idx].upper()}", (10, 30),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
                    cv2.putText(display, f"REPS: {rep_count}", (10, 70),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)
                    cv2.putText(display, "Show CLOSED FIST to STOP", (10, h - 60),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    if display_feedback:
                        cv2.putText(display, display_feedback, (10, h - 30),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
                    
                    frame = display
                else:
                    # No pose yet — keep UI up
                    cv2.putText(display, f"BICEP CURL - {STAGES[current_stage_idx].upper()}", (10, 30),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
                    cv2.putText(display, f"REPS: {rep_count}", (10, 70),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)
                    cv2.putText(display, "Waiting for pose detection...", (10, 110),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
                    cv2.putText(display, "Show CLOSED FIST to STOP", (10, h - 60),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    frame = display
            else:
                # Waiting for start gesture
                header = selected_exercise or "Unknown Exercise"
                draw_centered_text(frame, header, 40, base_scale=1.2, thickness=3, color=(0, 165, 255))
                cv2.putText(frame, "Waiting for START (open palm - 4+ fingers)", (40, 90),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                
                if hands_results.multi_hand_landmarks:
                    for hand_landmarks in hands_results.multi_hand_landmarks:
                        mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        cv2.putText(frame, "Press 'q' to quit", (10, h - 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200, 200, 200), 1)

        # display = cv2.addWeighted(frame, 0.55, np.zeros_like(frame), 0.45, 0)
        
        cv2.imshow("Workout Tracker", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    cap.release()
    cv2.destroyAllWindows()
    hands_detector.close()
    pose_detector.close()


C:\Users\chara\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\chara\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\chara\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\google\protobuf\symbol_database.py: